## CMSC Interactive Fiction Storytelling - DnD Project  
### LLM TO DETECTION PLAYER INTENT BASED ON PLAYER NARRATIVES AND GAME STATE
#### Team Members, Arya Honraopatil, Saksham Kumar Sharma, and Patty Delafuente  

Ablation study - M2 before utterances, combat state before, current actor state 

In [1]:
import os
import json
import numpy as np
from rouge_score import rouge_scorer, scoring

In [3]:
!ls /models/llama-3_1-8b-instruct-nemo_v1.0

ls: cannot access '/models/llama-3_1-8b-instruct-nemo_v1.0': No such file or directory


In [2]:
DATA_DIR = os.path.join("data")

!ls {DATA_DIR}

fireball_test.jsonl
fireball_test_finetuning-n128.jsonl
fireball_test_finetuning-n128.jsonl.idx.info
fireball_test_finetuning-n128.jsonl.idx.npy
fireball_test_finetuning-n128_m2.jsonl
fireball_test_finetuning-n128_m2.jsonl.idx.info
fireball_test_finetuning-n128_m2.jsonl.idx.npy
fireball_test_finetuning.jsonl
fireball_test_finetuning_m2.jsonl
fireball_test_finetuning_m3.jsonl
fireball_test_m2.jsonl
fireball_test_m3.jsonl
fireball_train.jsonl
fireball_train_finetuning.jsonl
fireball_train_finetuning.jsonl.idx.info
fireball_train_finetuning.jsonl.idx.npy
fireball_train_finetuning_m2.jsonl
fireball_train_finetuning_m2.jsonl.idx.info
fireball_train_finetuning_m2.jsonl.idx.npy
fireball_train_m2.jsonl
fireball_train_m3.jsonl
fireball_val.jsonl
fireball_val_finetuning.jsonl
fireball_val_finetuning.jsonl.idx.info
fireball_val_finetuning.jsonl.idx.npy
fireball_val_finetuning_m2.jsonl
fireball_val_finetuning_m2.jsonl.idx.info
fireball_val_finetuning_m2.jsonl.idx.npy
fireball_val_finetuning_m3.jso

In [5]:
TRAIN_DS = os.path.join(DATA_DIR, "fireball_train_finetuning_m5.jsonl")
VAL_DS = os.path.join(DATA_DIR, "fireball_val_finetuning_m5.jsonl")
TEST_DS = os.path.join(DATA_DIR, "fireball_test_finetuning_m5.jsonl")

In [6]:
!head -n1 {TRAIN_DS}

{"input": "You are a game agent in a DnD game. You will be provided with the current player and the in game player utterance. Your task is to predict what action, attack or spell the play intends. Then provide the AVRAE command that will initiate the players intention in a discord DnD game. Return the avrae command. Here is the game state. [CLS] Current Player: {actions: None, attacks: Fist, Regeneration, class: None, controller_id: 241419117855225963, description: None, effects: Aid, hp: <143/152 HP; Injured>, name: SG3, race: Shield Guardian, spells: }[SEP] Game State: Player: Cerberus (Fighter 7) (<74/84 HP; Injured> (+11 temp)) (Glaive, Halberd, Handaxe, Handaxe2, Moonrazor, Shortsword, Unarmed Strike, Polearm Master - Opportunity Attack) (Burning Hands, Flame Blade, Produce Flame) Player: lair () () () () Player: RS1 () (<22/22 HP; Healthy>) (Bite) () Player: RS2 () (<22/22 HP; Healthy>) (Bite) () Player: RS3 () (<22/22 HP; Healthy>) (Bite) () Player: RS4 () (<22/22 HP; Healthy>) 

In [7]:
!ls ./data/

fireball_test.jsonl
fireball_test_finetuning-n128.jsonl
fireball_test_finetuning-n128.jsonl.idx.info
fireball_test_finetuning-n128.jsonl.idx.npy
fireball_test_finetuning-n128_m2.jsonl
fireball_test_finetuning-n128_m2.jsonl.idx.info
fireball_test_finetuning-n128_m2.jsonl.idx.npy
fireball_test_finetuning.jsonl
fireball_test_finetuning_m2.jsonl
fireball_test_finetuning_m3.jsonl
fireball_test_m2.jsonl
fireball_test_m3.jsonl
fireball_train.jsonl
fireball_train_finetuning.jsonl
fireball_train_finetuning.jsonl.idx.info
fireball_train_finetuning.jsonl.idx.npy
fireball_train_finetuning_m2.jsonl
fireball_train_finetuning_m2.jsonl.idx.info
fireball_train_finetuning_m2.jsonl.idx.npy
fireball_train_finetuning_m3.jsonl
fireball_train_m2.jsonl
fireball_train_m3.jsonl
fireball_val.jsonl
fireball_val_finetuning.jsonl
fireball_val_finetuning.jsonl.idx.info
fireball_val_finetuning.jsonl.idx.npy
fireball_val_finetuning_m2.jsonl
fireball_val_finetuning_m2.jsonl.idx.info
fireball_val_finetuning_m2.jsonl.idx

In [8]:
!ls models

Llama-3.1-8B-Instruct  llama3_1_8b_instruct.nemo


In [2]:
%%bash

# Set paths to the model, train, validation and test sets.
MODEL="models/llama3_1_8b_instruct.nemo"

TRAIN_DS="[./data/fireball_train_finetuning_m5.jsonl]"
VALID_DS="[./data/fireball_val_finetuning_m5.jsonl]"
TEST_DS="[./data/fireball_test_finetuning_m5.jsonl]"
TEST_NAMES="[fireball]"

SCHEME="lora"
TP_SIZE=1
PP_SIZE=1
 
OUTPUT_DIR="./results/Meta-llama3.1-8B-Instruct-fb_m5"
rm -r $OUTPUT_DIR

torchrun --nproc_per_node=1 \
/opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_finetuning.py \
    exp_manager.exp_dir=${OUTPUT_DIR} \
    exp_manager.explicit_log_dir=${OUTPUT_DIR} \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    trainer.precision=bf16-mixed \
    trainer.val_check_interval=0.2 \
    trainer.max_steps=500 \
    model.data.train_ds.max_seq_length=4096 \
    model.data.validation_ds.max_seq_length=4096 \
    model.data.test_ds.max_seq_length=4096 \
    model.megatron_amp_O2=True \
    ++model.mcore_gpt=True \
    model.tensor_model_parallel_size=${TP_SIZE} \
    model.pipeline_model_parallel_size=${PP_SIZE} \
    model.micro_batch_size=1 \
    model.global_batch_size=32 \
    model.restore_from_path=${MODEL} \
    model.data.train_ds.file_names=${TRAIN_DS} \
    model.data.train_ds.concat_sampling_probabilities=[1.0] \
    model.data.validation_ds.file_names=${VALID_DS} \
    model.peft.peft_scheme=${SCHEME}

rm: cannot remove './results/Meta-llama3.1-8B-Instruct-fb_m5': No such file or directory
`zarr` distributed checkpoint backend is deprecated. Please switch to PyTorch Distributed format (`torch_dist`).
[NeMo W 2024-12-07 11:29:55 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    


[NeMo I 2024-12-07 11:29:55 megatron_gpt_finetuning:56] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-07 11:29:55 megatron_gpt_finetuning:57] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: bf16-mixed
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 500
      log_every_n_steps: 10
      val_check_interval: 0.2
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: ./results/Meta-llama3.1-8B-Instruct-fb_m5
      exp_dir: ./results/Meta-llama3.1-8B-Instruct-fb_m5
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.dat

[NeMo W 2024-12-07 11:29:55 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[NeMo I 2024-12-07 11:29:55 exp_manager:396] ExpManager schema
[NeMo I 2024-12-07 11:29:55 exp_manager:397] {'explicit_log_dir': None, 'exp_dir': None, 'name': None, 'version': None, 'use_datetime_version': True, 'resume_if_exists': False, 'resume_past_end': False, 'resume_ignore_no_checkpoint': False, 'resume_from_checkpoint': None, 'create_tensorboard_logger': True, 'summary_writer_kwargs': None, 'create_wandb_logger': False, 'wandb_logger_kwargs': None, 'create_mlflow_logger': False, 'mlflow_logger_kwargs': {'experiment_name': None, 'tracking_uri': None, 'tags': None, 'save_dir': './mlruns', 'prefix': '', 'artifact_location': None, 'run_id': None, 'log_model': False}, 'create_dllogger_logger': False, 'dllogger_logger_kwargs': {'verbose': False, 'stdout': False, 'json_file': './dllogger.json'}, 'create_clearml_logger': False, 'clearml_logger_kwargs': {'project': None, 'task': None, 'connect_pytorch': False, 'model_name': None, 'tags': None, 'log_model': False, 'log_cfg': False, 'log_

[NeMo E 2024-12-07 11:29:55 exp_manager:830] exp_manager received explicit_log_dir: ./results/Meta-llama3.1-8B-Instruct-fb_m5 and at least one of exp_dir: ./results/Meta-llama3.1-8B-Instruct-fb_m5, or version: None. Please note that exp_dir, name, and version will be ignored.
[NeMo W 2024-12-07 11:29:55 exp_manager:757] There were no checkpoints found in checkpoint_dir or no checkpoint folder at checkpoint_dir :results/Meta-llama3.1-8B-Instruct-fb_m5/checkpoints. Training from scratch.


...

Metric val_loss improved by 0.006 >= min_delta = 0.001. New best score: 1.483
Epoch 0, global step 500: 'validation_loss' reached 1.48343 (best 1.48343), saving model to '/workspace/nvdata/storytelling/project_691/results/Meta-llama3.1-8B-Instruct-fb_m5/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=1.483-step=500-consumed_samples=16000.0.ckpt' as top 1


Epoch 0: : 100%|██████████| 500/500 [2:16:27<00:00, reduced_train_loss=1.450, global_step=499.0, consumed_samples=1.6e+4, train_step_timing in s=3.010, val_loss=1.480][NeMo I 2024-12-07 13:47:02 nlp_overrides:593] Removing checkpoint: /workspace/nvdata/storytelling/project_691/results/Meta-llama3.1-8B-Instruct-fb_m5/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=1.489-step=400-consumed_samples=12800.0.ckpt
[NeMo I 2024-12-07 13:47:02 nlp_overrides:593] Removing checkpoint: /workspace/nvdata/storytelling/project_691/results/Meta-llama3.1-8B-Instruct-fb_m5/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=1.489-step=400-consumed_samples=12800.0-last.ckpt


`Trainer.fit` stopped: `max_steps=500` reached.


Epoch 0: : 100%|██████████| 500/500 [2:16:27<00:00, reduced_train_loss=1.450, global_step=499.0, consumed_samples=1.6e+4, train_step_timing in s=3.010, val_loss=1.480]


Restoring states from the checkpoint path at /workspace/nvdata/storytelling/project_691/results/Meta-llama3.1-8B-Instruct-fb_m5/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=1.483-step=500-consumed_samples=16000.0.ckpt
Restored all states from the checkpoint at /workspace/nvdata/storytelling/project_691/results/Meta-llama3.1-8B-Instruct-fb_m5/checkpoints/megatron_gpt_peft_lora_tuning--validation_loss=1.483-step=500-consumed_samples=16000.0.ckpt


In [3]:
#|!ls -l ./results/Meta-llama3.1-8B-Instruct-fb/checkpoints
!ls -l ./results/Meta-llama3.1-8B-Instruct-fb_m5

total 384
drwxr-xr-x 2 root root   4096 Dec  7 13:47 checkpoints
-rw-r--r-- 1 root root    919 Dec  7 11:29 cmd-args.log
-rw-r--r-- 1 root root 286450 Dec  7 13:47 events.out.tfevents.1733571030.61817bf2ac66.2561.0
-rw-r--r-- 1 root root   5971 Dec  7 11:30 hparams.yaml
-rw-r--r-- 1 root root   3084 Dec  7 13:47 lightning_logs.txt
-rw-r--r-- 1 root root  27893 Dec  7 11:58 nemo_error_log.txt
-rw-r--r-- 1 root root  51358 Dec  7 13:47 nemo_log_globalrank-0_localrank-0.txt


In [4]:
 #Create a smaller test subset for a quick eval demonstration.

!head -n 128 ./data/fireball_test_finetuning_m5.jsonl > ./data/fireball_test_finetuning-n128_m5.jsonl

In [5]:
%%bash
MODEL="models/llama3_1_8b_instruct.nemo"
#MODEL="/workspace/llama-3_1-8b-instruct-nemo_v1.0/llama3_1_8b_instruct.nemo"



TEST_DS="[./data/fireball_test_finetuning-n128_m5.jsonl]" # Smaller test split
# TEST_DS="[./curated-data/law-qa-test_preprocessed.jsonl]" # Full test set
TEST_NAMES="[fireball]"

TP_SIZE=1
PP_SIZE=1

# This is where your LoRA checkpoint was saved
PATH_TO_TRAINED_MODEL="results/Meta-llama3.1-8B-Instruct-fb_m5/checkpoints/megatron_gpt_peft_lora_tuning.nemo"

# The generation run will save the generated outputs over the test dataset in a file prefixed like so
OUTPUT_PREFIX="fb_lora_m5"
#global batch size (1) is not divisible by micro batch size (1) times data parallel size (2)
torchrun --nproc_per_node=1
python /opt/NeMo/examples/nlp/language_modeling/tuning/megatron_gpt_generate.py \
    model.restore_from_path=${MODEL} \
    model.peft.restore_from_path=${PATH_TO_TRAINED_MODEL} \
    trainer.devices=1 \
    trainer.num_nodes=1 \
    model.data.test_ds.file_names=${TEST_DS} \
    model.data.test_ds.names=${TEST_NAMES} \
    model.data.test_ds.global_batch_size=16 \
    model.data.test_ds.micro_batch_size=1 \
    model.data.test_ds.tokens_to_generate=25 \
    model.tensor_model_parallel_size=${TP_SIZE} \
    model.pipeline_model_parallel_size=${PP_SIZE} \
    inference.greedy=True  \
    model.data.test_ds.output_file_path_prefix=${OUTPUT_PREFIX} \
    model.data.test_ds.write_predictions_to_file=True \
    model.data.test_ds.truncation_field="null" \
    model.data.test_ds.add_bos=False \
    model.data.test_ds.add_eos=True \
    model.data.test_ds.add_sep=False \
    model.data.test_ds.label_key="output" \
    model.data.test_ds.prompt_template="\{input\}\ \{output\}"

usage: torchrun [-h] [--nnodes NNODES] [--nproc-per-node NPROC_PER_NODE]
                [--rdzv-backend RDZV_BACKEND] [--rdzv-endpoint RDZV_ENDPOINT]
                [--rdzv-id RDZV_ID] [--rdzv-conf RDZV_CONF] [--standalone]
                [--max-restarts MAX_RESTARTS]
                [--monitor-interval MONITOR_INTERVAL]
                [--start-method {spawn,fork,forkserver}] [--role ROLE] [-m]
                [--no-python] [--run-path] [--log-dir LOG_DIR] [-r REDIRECTS]
                [-t TEE] [--local-ranks-filter LOCAL_RANKS_FILTER]
                [--node-rank NODE_RANK] [--master-addr MASTER_ADDR]
                [--master-port MASTER_PORT] [--local-addr LOCAL_ADDR]
                training_script ...
torchrun: error: the following arguments are required: training_script, training_script_args
`zarr` distributed checkpoint backend is deprecated. Please switch to PyTorch Distributed format (`torch_dist`).
[NeMo W 2024-12-07 14:01:04 nemo_logging:349] /usr/local/lib/python3.10/d

[NeMo I 2024-12-07 14:01:04 megatron_gpt_generate:125] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-12-07 14:01:04 megatron_gpt_generate:126] 
    name: megatron_gpt_peft_${model.peft.peft_scheme}_tuning
    trainer:
      devices: 1
      accelerator: gpu
      num_nodes: 1
      precision: 16
      logger: false
      enable_checkpointing: false
      use_distributed_sampler: false
      max_epochs: 9999
      max_steps: 20000
      log_every_n_steps: 10
      val_check_interval: 200
      gradient_clip_val: 1.0
    exp_manager:
      explicit_log_dir: null
      exp_dir: null
      name: ${name}
      create_wandb_logger: false
      wandb_logger_kwargs:
        project: null
        name: null
      resume_if_exists: true
      resume_ignore_no_checkpoint: true
      create_checkpoint_callback: true
      checkpoint_callback_params:
        monitor: validation_${model.data.test_ds.metric.name}
        save_top_k: 1
        mode: max
        save_nemo_o

[NeMo W 2024-12-07 14:01:04 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/_graveyard/precision.py:49: The `MixedPrecisionPlugin` is deprecated. Use `pytorch_lightning.plugins.precision.MixedPrecision` instead.
    
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[NeMo W 2024-12-07 14:01:09 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-07 14:01:09 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-12-07 14:01:09 megatron_base_model:1182] The model: MegatronGPTSFTModel() does not have field.name: moe_extended_tp in its cfg. Add this key to cfg or config_mapping to make to make it 

[NeMo I 2024-12-07 14:01:09 megatron_init:269] Rank 0 has data parallel group : [0]
[NeMo I 2024-12-07 14:01:09 megatron_init:275] Rank 0 has combined group of data parallel and context parallel : [0]
[NeMo I 2024-12-07 14:01:09 megatron_init:280] All data parallel group ranks with context parallel combined: [[0]]
[NeMo I 2024-12-07 14:01:09 megatron_init:283] Ranks 0 has data parallel rank: 0
[NeMo I 2024-12-07 14:01:09 megatron_init:291] Rank 0 has context parallel group: [0]
[NeMo I 2024-12-07 14:01:09 megatron_init:294] All context parallel group ranks: [[0]]
[NeMo I 2024-12-07 14:01:09 megatron_init:295] Ranks 0 has context parallel rank: 0
[NeMo I 2024-12-07 14:01:09 megatron_init:302] Rank 0 has model parallel group: [0]
[NeMo I 2024-12-07 14:01:09 megatron_init:303] All model parallel group ranks: [[0]]
[NeMo I 2024-12-07 14:01:09 megatron_init:312] Rank 0 has tensor model parallel group: [0]
[NeMo I 2024-12-07 14:01:09 megatron_init:316] All tensor model parallel group ranks: 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Testing DataLoader 0:   0%|          | 0/8 [00:00<?, ?it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  12%|█▎        | 1/8 [00:11<01:19,  0.09it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  25%|██▌       | 2/8 [00:25<01:16,  0.08it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  38%|███▊      | 3/8 [00:34<00:58,  0.09it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  50%|█████     | 4/8 [00:42<00:42,  0.09it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  62%|██████▎   | 5/8 [00:57<00:34,  0.09it/s]setting number of micro-batches to constant 1
setting number of micro-batches to constant 16
Testing DataLoader 0:  75%|███████▌  | 6/8 [01:0

[NeMo W 2024-12-07 14:03:07 megatron_gpt_sft_model:642] No training data found, reconfiguring microbatches based on validation batch sizes.
[NeMo W 2024-12-07 14:03:07 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    


setting number of micro-batches to constant 16


[NeMo W 2024-12-07 14:03:07 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('test_loss_fireball', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    
[NeMo W 2024-12-07 14:03:07 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:439: It is recommended to use `self.log('test_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
    


Testing DataLoader 0: 100%|██████████| 8/8 [01:26<00:00,  0.09it/s]
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     1.48872971534729      │
│    test_loss_fireball     │     1.48872971534729      │
│         val_loss          │     1.48872971534729      │
└───────────────────────────┴───────────────────────────┘


In [6]:
# Take a look at predictions
!head -n1  fb_lora_m5_test_fireball_inputs_preds_labels.jsonl

{"input": "Utterances : [Keeps shredding minion and will move 60ft up to be at 160total][SEP] \nAvrae Command:", "pred": " [!a \"longsword\" -t \"minion1\" -d 1d6][SEP]", "label": " [!i a bite -t min][SEP]"}


In [7]:
def compute_rouge(input_file: str) -> dict:
    ROUGE_KEYS = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    scorer = rouge_scorer.RougeScorer(ROUGE_KEYS, use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()
    lines = [json.loads(line) for line in open(input_file)]
    num_response_words = []
    num_ref_words = []
    for idx, line in enumerate(lines):
        prompt = line['input']
        response = line['pred']
        answer = line['label']
        scores = scorer.score(response, answer)
        aggregator.add_scores(scores)
        num_response_words.append(len(response.split()))
        num_ref_words.append(len(answer.split()))

    result = aggregator.aggregate()
    rouge_scores = {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
    print(rouge_scores)
    print(f"Average and stddev of response length: {np.mean(num_response_words):.2f}, {np.std(num_response_words):.2f}")
    print(f"Average and stddev of ref length: {np.mean(num_ref_words):.2f}, {np.std(num_ref_words):.2f}")

    return rouge_scores

In [8]:
compute_rouge("./fb_lora_m5_test_fireball_inputs_preds_labels.jsonl")

{'rouge1': 42.6937, 'rouge2': 12.0913, 'rougeL': 42.2681, 'rougeLsum': 42.2012}
Average and stddev of response length: 5.17, 1.93
Average and stddev of ref length: 12.28, 22.03


{'rouge1': 42.6937, 'rouge2': 12.0913, 'rougeL': 42.2681, 'rougeLsum': 42.2012}